In [1]:
import tensorflow as tf

In [100]:
gHidden1OutputSize = 1
pHidden1OutputSize = 50
sHidden1OutputSize = 50
hidden2OutputSize = 50
seed = 0

In [101]:
tf.reset_default_graph()
#global variables
with tf.variable_scope('GlobalVariables'):
    side2move = tf.placeholder(tf.float32, shape=[None, 1], name='Side2Move')#1 bit (everything is really just floats)
    castlingRights = tf.placeholder(tf.float32, shape=[None, 4], name='CastlingRights')#4 bits
    numOfEachPieceType = tf.placeholder(tf.float32, shape=[None, 10], name = 'NumOfEachPieceType')#array of 10 floats, excluding kings (should these be normalized? I think so)

    globalFeats = tf.concat([side2move, castlingRights, numOfEachPieceType], 1, name='GlobalFeatures')

    #global hidden layer
    #gWeights1 =  self.createWeight('Weights 1')
    #gBais1 = self.createWeight('Bias 1')
    gHidden1 = tf.layers.dense(globalFeats, gHidden1OutputSize, name='GlobalHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))
    #testWeights = tf.get_variable("kernel")
#piece-centric variables
with tf.variable_scope('Piece-CentricVariables'):
    pieceIsPresent = tf.placeholder(tf.float32, shape=[None, 1], name='PieceIsPresent') # array of 48 bits
    xyPosition = tf.placeholder(tf.float32, shape=[None, 48,2], name='XYPosition') #array of 48 by 2 normalized floats (choose -1 if not present?)
    xyPositionFlat = tf.reshape(xyPosition, shape=[-1, 48*2])
    pieceLowValAttacker = tf.placeholder(tf.float32, shape=[None, 48, 12], name='PieceLowValAttacker') #array of 48 pieces and a one-hot vector representing 12 possible attackers
    pieceLowValAttackerFlat = tf.reshape(pieceLowValAttacker, shape=[-1, 48*12])
    pieceLowValDefender = tf.placeholder(tf.float32, shape=[None, 48, 12], name='PieceLowValDefender') #array of 48 pieces and a one-hot vector representing 12 possible defenders
    pieceLowValDefenderFlat = tf.reshape(pieceLowValDefender, shape=[-1, 48*12])
    maxTravelDistance = tf.placeholder(tf.float32, shape=[None, 14]) #array of 14 floats, normalized (one for each sliding piece)
    movePieceType = tf.placeholder(tf.float32, shape=[None, 6]) #one hot array of size 6
    pawnPromoType =  tf.placeholder(tf.float32, shape=[None, 7]) #one hot array of size 7 (extra spot for when there is no pawn promotion, or should the vector be zero then?)

    pieceFeats = tf.concat([pieceIsPresent, xyPositionFlat, pieceLowValAttackerFlat, 
        pieceLowValDefenderFlat, maxTravelDistance, movePieceType, 
        pawnPromoType], 1, name='Piece-CentricFeatures')

    #piece-centric hidden layer
    #gWeights1 =  self.createWeight('Weights 1')
    #gBais1 = self.createWeight('Bias 1')
    pHidden1 = tf.layers.dense(pieceFeats, pHidden1OutputSize, name='GlobalHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))

#square-centric variables
with tf.variable_scope('Sqaure-CentricVariables'):
    #array of 12 by 12 squares and a one-hot vector representing 12 possible attackers
    squareLowValAttacker = tf.placeholder(tf.float32, shape=[None, 12, 12, 12], name='SquareLowValueAttacker')
    squareLowValAttackerFlat = tf.reshape(squareLowValAttacker, shape=[-1, 12*12*12])
    #array of 12 by 12 squares and a one-hot vector representing 12 possible defenders
    squareLowValDefender = tf.placeholder(tf.float32, shape=[None, 12, 12, 12], name='SquareLowValueDefender')
    squareLowValDefenderFlat = tf.reshape(squareLowValAttacker, shape=[-1, 12*12*12])
    #two floats (x and y)
    moveStartingSquare = tf.placeholder(tf.float32, shape=[None, 2], name='MoveStartingSquare')
    #two floats (x and y)
    moveEndingSquare = tf.placeholder(tf.float32, shape=[None, 2], name='MoveEndingSquare')
    
    squareFeats = tf.concat([squareLowValAttackerFlat, squareLowValDefenderFlat, moveStartingSquare, moveEndingSquare], 1)

    #square-centric hidden layer
    sHidden1 = tf.layers.dense(pieceFeats, sHidden1OutputSize, name='SquareHiddenLayer1', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))

input2 = tf.concat([gHidden1, pHidden1, sHidden1], 1)
hidden2 = tf.layers.dense(input2, hidden2OutputSize, name='HiddenLayer2', activation=tf.nn.relu, kernel_initializer=tf.contrib.layers.variance_scaling_initializer(seed=seed))


In [48]:
squareFeats

<tf.Tensor 'Sqaure-CentricVariables/concat:0' shape=(?, 3460) dtype=float32>

In [96]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #print sess.run( weights = tf.get_variable("GlobalVariable/kernel"))
    print sess.run(gHidden1, feed_dict={side2move:[[0], 
                                                   [1]], 
                                           castlingRights:[[1,1, 1, 1], 
                                                           [1,1, 1, 1]], 
                                           numOfEachPieceType:[[0, 0, 0, 1, 0, 1, 0],
                                                               [0, 1, 0, 1, 0, 1, 0]]})

[[ 0.07309246]
 [ 0.3627401 ]]


In [66]:
dir(gHidden1)

['OVERLOADABLE_OPERATORS',
 '__abs__',
 '__add__',
 '__and__',
 '__array_priority__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__div__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__invert__',
 '__iter__',
 '__le__',
 '__lt__',
 '__mod__',
 '__module__',
 '__mul__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_add_consumer',
 '_as_node_def_input',
 '_consumers',
 '_dtype',
 '_handle_dtype',
 '_handle_shape',
 '_op',
 '_override_operator',
 '_shape',
 '_shape_as_list',
 '_value_index',
 'consumers',
 'device',
 'dtype',
 'eval',
 'get_shape'

In [75]:
dir(gHidden1.graph)

['_ControlDependenciesController',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_function',
 '_add_op',
 '_apply_device_functions',
 '_as_graph_def',
 '_as_graph_element_locked',
 '_attr_scope',
 '_attr_scope_map',
 '_building_function',
 '_check_not_finalized',
 '_collections',
 '_colocation_stack',
 '_container',
 '_control_dependencies_for_inputs',
 '_control_dependencies_stack',
 '_control_flow_context',
 '_current_control_dependencies',
 '_default_original_op',
 '_device_function_stack',
 '_finalized',
 '_functions',
 '_get_control_flow_context',
 '_get_function',
 '_gradient_override_map',
 '_graph_def_versions',
 '_handle_deleters',
 '_handle_feeders',
 '_handle_movers',
 '_handle_readers',
 '_is_function',
 '_kernel_label_map',
 '_last_id',
 '_lock',


In [76]:
print(tf.__version__)

1.0.1
